In [1]:
from dotenv import load_dotenv
import os
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [2]:
load_dotenv()
API_KEY = os.getenv('OPENAI_API_KEY')
Model = 'gpt-3.5-turbo'

In [3]:
model = ChatOpenAI(api_key=API_KEY,model=Model, temperature=0.2)
model.invoke('What is coding?')

AIMessage(content='Coding is the process of creating instructions for a computer to follow in order to perform a specific task or function. It involves writing lines of code using programming languages such as Python, Java, or C++, which are then translated by the computer into actions or outputs. Coding is used to develop software, websites, apps, and other digital technologies. It requires logical thinking, problem-solving skills, and attention to detail.', response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 11, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c3273353-0b71-44e5-9785-60e7e6fa7a16-0', usage_metadata={'input_tokens': 11, 'output_tokens': 82, 'total_tokens': 93})

In [4]:
parser = StrOutputParser()
chain = model|parser

In [5]:
def format_text(text):
    # Add extra line breaks for better readability
    formatted_text = text.replace('\n', '\n\n')
    return formatted_text

In [6]:
formatted_output1 = format_text(chain.invoke('What is coding?'))
print(formatted_output1)

Coding is the process of creating instructions for a computer to follow in order to perform a specific task. It involves writing and organizing lines of code in a programming language that the computer can understand and execute. Coding is used to create software, websites, mobile apps, and other digital technologies. It requires logical thinking, problem-solving skills, and attention to detail.


In [7]:
file_loader = PyPDFLoader('Hitachi Manual.pdf')
page = file_loader.load_and_split()
len(page)

32

In [8]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
pages = splitter.split_documents(page)
pages[4]

Document(page_content="the IFEEL  function will be shut of f. \nIf the IFEEL  function can't be shut of f, please try\nto press this button about 5 seconds.\nAdvice to put the remote controller in the place\nwhere the indoor unit receive signal easily .\nAdvice to cancel the IFEEL  mode so as to save\nenergy when stopping the air conditioner .   ECO BUTT ON \n   CLEAN BUTT ON 14\nUsed to stop or start Horizontal adjustment \nlouver swinging and set the desired left/right\nairflow direction.   SWING BUTT ON \n1     indicator MuteEconomy  indicator\nIfeel\nLockNanoe  indicator\nTurbo indicator\nAirflow up and down  indicator Airflow left and right  indicator15 4\n1010\n13 133 31 1\n2 2\n6 6\n5 7 4 14\n171711 11\n121295\n14 8 79\n16168 15TEMP\nLOCK\nPress them together for 3 seconds to start or stop \n.LOCK\noperation+1211RTY2", metadata={'source': 'Hitachi Manual.pdf', 'page': 2})

In [9]:
vector_storage = FAISS.from_documents(pages, OpenAIEmbeddings())
retriever = vector_storage.as_retriever()

In [10]:
question_template = """
You're a smart chatbot that answers user generated prompts only based on the context given to you.
You don't make any assumptions.
context:{context}
question:{question}
"""

In [11]:
prompt = PromptTemplate.from_template(template=question_template)
print(prompt.format(context = ' Here is the context to use',
              question = ' Answer this question based on the context'))


You're a smart chatbot that answers user generated prompts only based on the context given to you.
You don't make any assumptions.
context: Here is the context to use
question: Answer this question based on the context




In [12]:
result = RunnableParallel(context = retriever, question = RunnablePassthrough())

In [13]:
chain = result | prompt | model | parser

In [14]:
chain.invoke('what is coding?')

'I\'m sorry, but based on the context provided, there is no information about "coding" in the documents. The documents mainly discuss the operation instructions and safety precautions for a Hitachi appliance.'

In [15]:
chain.invoke('What is the title of this pdf?')

'The title of this PDF document is "Hitachi Manual.pdf".'

In [16]:
retriever.invoke('what is the title of this pdf?')

[Document(page_content='Specifications  in this document  are subject to change  without notice, in order that Hitachi -Johnson  Controls Air Conditioning , Inc. may \nbring  the latest innovations  to their customers.\n© 2022 Hitachi -  Johnson Controls  Air Conditioning , Inc. 2022 .05       Version.A      Printed in  ChinaHitachi-Johnson Controls Air Conditioning , Inc.', metadata={'source': 'Hitachi Manual.pdf', 'page': 21}),
 Document(page_content='available such as the operating manual or \ninstallation manual.WARNING\nWARNING\nCAUTION\nCAUTION\nCAUTION\n13', metadata={'source': 'Hitachi Manual.pdf', 'page': 14}),
 Document(page_content='the wal  \nshould be \nover 250mml\nover 250mmInstallation instructions \n    Explanation of symbols displayed on the indoor \n    unit or outdoor unit.\nCaution, risk of fire\nWaring;low burning\nvelocity materialThis symbol shows that this appliance uses\na flammable refrigerant.\nIf the refrigerant is leaked and exposed to \nan external igniti

In [17]:
chain.invoke('What are the key safety instructions to be followed?')

'Some key safety instructions to be followed include:\n1. Ensuring that no electrical supply is connected to a circuit with a fault that compromises safety until it is satisfactorily dealt with.\n2. Using an adequate temporary solution if a fault cannot be corrected immediately but operation needs to continue.\n3. Discharging capacitors in a safe manner to avoid sparking.\n4. Ensuring that there are no live electrical components and exposed wiring while charging, recovering, or purging the system.\n5. Checking for continuity of earth bonding.\n6. Disconnecting all electrical supplies from equipment before repairing sealed components.\n7. Instructing maintenance staff on the nature of work being carried out and avoiding work in confined spaces.\n8. Checking for the presence of refrigerant with an appropriate detector before and during work.\n9. Ensuring the leak detection equipment used is suitable for flammable refrigerants.\n10. Having appropriate fire extinguishing equipment availabl

In [18]:
chain.invoke('What are the installation instructions, give in detail as to help with the installation of an indoor unit.')

'The installation instructions for the indoor unit are as follows:\n\n1. Select the installation locations:\n- Choose a location where there is no obstacle near the air outlet and air can be easily blown to every corner.\n- Ensure that piping and wall holes can be easily arranged.\n- Maintain the required space from the unit to the ceiling and wall according to the installation diagram.\n- Ensure easy removal of the air filter.\n- Keep the unit and remote controller at least 1m apart from television, radio, etc.\n- Keep the unit away from fluorescent lamps.\n- Do not obstruct the air inlet with any objects.\n- Install the unit on a strong wall that can bear its weight.\n- Choose a place that will not increase operation noise and vibration.\n- Keep the unit away from direct sunlight, heating sources, flammable materials, and combustion apparatuses.\n- The pipe length should be 15 meters maximum, and the height should be less than 5m.\n\n2. Installing the Mounting Plate:\n- Decide on an 

In [19]:
formatted_output2 = format_text(chain.invoke('What are the installation instructions, give in detail as to help with the installation of an indoor unit.'))
print(formatted_output2)

The installation instructions for the indoor unit are as follows:



1. Select the installation locations:

   - Choose a location where there is no obstacle near the air outlet and air can be easily blown to every corner.

   - Ensure that piping and wall holes can be easily arranged.

   - Maintain the required space from the unit to the ceiling and wall according to the installation diagram.

   - Ensure easy removal of the air filter.

   - Keep the unit and remote controller at least 1m apart from television, radio, etc.

   - Keep the unit away from fluorescent lamps.

   - Do not obstruct the air inlet with any objects.

   - Install the unit on a strong wall that can bear its weight.

   - Install in a place that will not increase operation noise and vibration.

   - Keep the unit away from direct sunlight, heating sources, flammable materials, and combustion apparatuses.



2. Indoor unit installation:

   - Install the mounting plate horizontally at the decided location.

   

In [20]:
chain.invoke('Give me details about eco mode.')

'In Eco mode, the air conditioner operates with low noise performance and provides energy-saving performance by running on lower running currency. You can start the Sleep mode at the same time in Eco mode. To cancel Eco mode, you can press the MODE, FAN SPEED, SMART, or SUPER button. Please note that the Economy button is ineffective in SMART and SUPER mode.'